In [2]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib import cm

In [3]:
# Matrix preparation auxiliary functions

def prep_diags(N, d):
    # This performs the delta(x-x'+da) operation for the matrix
    if d == 0:
        return np.eye(N, dtype=complex)
    else:
        mat = np.zeros((N, N), dtype=complex)
        if d == 1:
            for i in range(N - 1):
                mat[i + 1][i] = 1
        else: # d == - 1
            for i in range(N - 1):
                mat[i][i + 1] = 1
        return mat

def mat2hext(mat, q):
    N = mat.shape[0]
    board = np.zeros((4 * N, 4 * N), dtype=complex)
    board[((q // 4) * N):((q // 4 + 1) * N), ((q % 4) * N):((q % 4 + 1) * N)] = mat
    return board

def dot_pauli(mat, pauli_idxs):
    board = np.zeros((4 * mat.shape[0], 4 * mat.shape[0]), dtype=complex)
    if pauli_idxs == 'x, 1':
        board += mat2hext(mat, 2) + mat2hext(mat, 7) + mat2hext(mat, 8) + mat2hext(mat, 13)
    elif pauli_idxs == 'z, 1':
        board += mat2hext(mat, 0) + mat2hext(mat, 5) - mat2hext(mat, 10) - mat2hext(mat, 15)
    elif pauli_idxs == 'z, x':
        board += mat2hext(mat, 1) + mat2hext(mat, 4) - mat2hext(mat, 11) - mat2hext(mat, 14)
    elif pauli_idxs == 'z, y':
        board += -1j * mat2hext(mat, 1) + 1j * mat2hext(mat, 4) + 1j * mat2hext(mat, 11) - 1j * mat2hext(mat, 14)
    return np.matrix(board)

def fourier_ham(N, b, M, ky, t):
    return b * np.sin(ky) * dot_pauli(prep_diags(N, 0), 'z, x') + \
           b * (-1 / 2j) * dot_pauli(prep_diags(N, 1) - prep_diags(N, -1), 'z, y') + \
           dot_pauli((M - 2 * t * np.cos(ky)) * prep_diags(N, 0) - t * (prep_diags(N, 1) + prep_diags(N, -1)), 'x, 1')


In [41]:
# Predef

M = 2
N = 40
ks = np.linspace(-np.pi, np.pi, N, endpoint=False)

eigvls = np.zeros((len(ks), 4 * N), dtype=complex)
eigvs = np.zeros((len(ks), 4 * N, 4 * N), dtype=complex)

for iy, ky in enumerate(ks):
    hamiltonian = fourier_ham(N, 1, M, ky, 1)
    # According to documentation, the eigenvectors are sorted by eigenvalue, so the last one is the + band vector
    eigvls[iy, :], cur_eigvs = np.linalg.eigh(hamiltonian)
    eigvs[iy, :, :] = cur_eigvs

for band in range(4 * N):
    print(eigvls[:, band])
    plt.plot(ks, eigvls[:, band])
plt.title(f"Band Structure for M=2t")
plt.ylabel("Energy [arb. units]")
plt.xlabel("ky")
plt.show()

In [42]:
X, Y = np.meshgrid(np.arange(0, N, 1), ks)

fig, axs = plt.subplots(2, 2, subplot_kw={"projection": "3d"})
fig.suptitle(f"Eigenfunctions for M={M}")

surf = axs[0, 0].plot_surface(X, Y, np.abs(eigvs[:, :N, 2*N-1]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[0, 0].set_xlabel('x (site)')
axs[0, 0].set_ylabel('ky')
axs[0, 0].title.set_text('Sublattice A, spin up')

surf = axs[0, 1].plot_surface(X, Y, np.abs(eigvs[:, N:2*N, 2*N-1]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[0, 1].set_xlabel('x (site)')
axs[0, 1].set_ylabel('ky')
axs[0, 1].title.set_text('Sublattice A, spin down')

surf = axs[1, 0].plot_surface(X, Y, np.abs(eigvs[:, 2*N:3*N, 2*N-1]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[1, 0].set_xlabel('x (site)')
axs[1, 0].set_ylabel('ky')
axs[1, 0].title.set_text(f"Sublattice B, spin up")

surf = axs[1, 1].plot_surface(X, Y, np.abs(eigvs[:, 3*N:, 2*N-1]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[1, 1].set_xlabel('x (site)')
axs[1, 1].set_ylabel('ky')
axs[1, 1].title.set_text(f"Sublattice B, spin down")

plt.show()